# quant-econ Solutions: Markov Asset Pricing

Solutions for http://quant-econ.net/jl/markov_asset.html

Let's do some standard imports.

In [1]:
using QuantEcon
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

Next we load the code from the lectures into a cell to have access to the functions.  This next line assumes that you have the file `asset_pricing.jl` from [QuantEcon.applications](https://github.com/QuantEcon/QuantEcon.applications) in the same directory as this notebook.

In [2]:
include("asset_pricing.jl")  #  Assumes you have this file in the same directory as this notebook

call_option (generic function with 2 methods)

## Exercise 1

In [3]:
n = 5
P = 0.0125 .* ones(n, n)
P = P .+ diagm(0.95 - 0.0125 .* ones(5))
s = [0.95, 0.975, 1.0, 1.025, 1.05]  # state values
mc = MarkovChain(P, s)

gamma = 2.0
beta = 0.94
zeta = 1.0
p_s = 150.0

150.0

Next we'll create an instance of `AssetPriceModel` to feed into the functions.

In [4]:
ap = AssetPriceModel(beta=beta, mc=mc, gamma=gamma, g=x -> x)

AssetPriceModel(0.94,2.0,Discrete Markov Chain
stochastic matrix of type Array{Float64,2}:
5x5 Array{Float64,2}:
 0.95    0.0125  0.0125  0.0125  0.0125
 0.0125  0.95    0.0125  0.0125  0.0125
 0.0125  0.0125  0.95    0.0125  0.0125
 0.0125  0.0125  0.0125  0.95    0.0125
 0.0125  0.0125  0.0125  0.0125  0.95  ,5,(anonymous function))

In [5]:
v = tree_price(ap)
println("Lucas Tree Prices: $v\n")

Lucas Tree Prices: [29.474015777145,21.935706611219704,17.571422357262907,14.725150017725886,12.722217630644252]



In [6]:
v_consol = consol_price(ap, 1.0)
println("Consol Bond Prices: $(v_consol)\n")

Consol Bond Prices: [753.8710047641985,242.5514408198945,148.67554548466475,109.2510896502471,87.5686013853112]



In [7]:
w = call_option(ap, zeta, p_s)

5x1 Array{Float64,2}:
 603.871 
 176.839 
 108.677 
  80.0518
  64.3084

## Exercise 2

Here's a suitable function:

In [8]:
function finite_horizon_call_option(ap::AssetPriceModel, 
                                    zeta::Float64, 
                                    p_s::Float64, 
                                    k::Int)

    # == Simplify names, set up matrices  == #
    beta, gamma, P, y = ap.beta, ap.gamma, ap.mc.p, ap.mc.state_values
    y = reshape(y, 1, ap.n)
    M = P .* ap.g(y).^(- gamma)

    # == Make sure that a unique console price exists == #
    test_stability(ap, M)

    # == Compute option price == #
    p = consol_price(ap, zeta)
    w = zeros(ap.n, 1)
    for i in 1:k
        # == Maximize across columns == #
        w = max(beta * M * w, p - p_s)
    end

    return w
end

finite_horizon_call_option (generic function with 1 method)

In [9]:
lines = []
labels = []
for k in [5, 25]
    w = finite_horizon_call_option(apm, zeta, p_s, k)
    push!(lines, w)
    push!(labels, "k = $k")
end
plot(lines, labels=labels')

LoadError: LoadError: UndefVarError: apm not defined
while loading In[9], in expression starting on line 3

Not surprisingly, the option has greater value with larger $k$.  This is because the owner has a longer time horizon over which he or she may exercise the option.